In [1]:
import numpy as np 
#import matplotlib.pyplot as plt
import math # for tan

#from PIL import Image # Python Imaging Library (Fork)

In [2]:
def calcmoments(img):
    # function to calculate moments
    # m00 = 0; m01 = 0; m10 = 0; m11 = 0; m20 = 0; m02 = 0
    moments = np.zeros(6, dtype=int)
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if (img[y,x] & 128): # check if 7th bit is '1', it is set for all gray values >=128
                moments[0] += 1
                moments[1] += y
                moments[2] += x
                moments[3] += x*y
                moments[4] += x*x
                moments[5] += y*y
    
    return moments
    
    
def chalctheta(m):
    # calculate cX,cY,theta of blob
    cxytheta = np.zeros(3, dtype=float   )
    cxytheta[0] = int(m[2]/m[0])
    cxytheta[1] = int(m[1]/m[0])
    mu20 = (m[4]/m[0])-cxytheta[0]**2
    mu11 = (m[3]/m[0])-cxytheta[0]*cxytheta[1]
    mu02 = (m[5]/m[0])-cxytheta[1]**2
    cxytheta[2] = 0.5 *  math.atan2( (2*mu11) , (mu20-mu02) )
    
    return cxytheta


In [3]:
import cv2

for i in np.arange(15):
    img = cv2.imread('Block'+str(i)+'.png',0)
    
    m = calcmoments(img)
    #print("Moments: m00: {} m01: {} m10: {} m11: {} m20: {} m02{: }".format(m[0],m[1],m[2],m[3],m[4],m[5]))
    
    cxytheta = chalctheta(m)
    print('Degree clockwise:',np.degrees(cxytheta[2]),'°')
    
    # visu
    length = 130
    x1 = cxytheta[0] + length * math.cos(cxytheta[2])
    y1 = cxytheta[1] + length * math.sin(cxytheta[2]) 
    x2 = cxytheta[0] - length * math.cos(cxytheta[2])
    y2 = cxytheta[1] - length * math.sin(cxytheta[2]) 

    cv2.namedWindow("main", cv2.WINDOW_NORMAL)
    color_image = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    lineThickness = 2
    cv2.line(color_image, (int(x1), int(y1)), (int(x2), int(y2)), (0,255,0), lineThickness)
    cv2.circle(color_image, (int(cxytheta[0]), int(cxytheta[1])), 5, (255, 0, 255), -1)
    cv2.putText(color_image, "centroid", (int(cxytheta[0]) - 25, int(cxytheta[1]) - 25),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 2)
    # display the image
    cv2.imshow("main", color_image)
    cv2.waitKey(0)

    cv2.destroyAllWindows()

Degree counterclockwise: 27.65995190709288 °
Degree counterclockwise: 47.24221817779009 °
Degree counterclockwise: 61.61106986695035 °
Degree counterclockwise: -85.07506331422528 °
Degree counterclockwise: -56.48139182186234 °
Degree counterclockwise: -23.59325437709572 °
Degree counterclockwise: -2.4050003610934474 °
Degree counterclockwise: 16.079079590816693 °
Degree counterclockwise: 30.1221522865163 °
Degree counterclockwise: 50.82384401598178 °
Degree counterclockwise: 71.24800388126877 °
Degree counterclockwise: -70.44264730759554 °
Degree counterclockwise: -35.19873593083016 °
Degree counterclockwise: -9.306453674431483 °
Degree counterclockwise: 12.698239761517945 °
